Importing the libraries

In [28]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from datetime import datetime as dt
import itertools

%matplotlib inline

Read the data from all files

In [29]:
# Read data from the CSV into a dataframe
folder='./Datasets/'
raw_data_1 = pd.read_csv(folder +'2000-01.csv')
raw_data_2 = pd.read_csv(folder +'2001-02.csv')
raw_data_3 = pd.read_csv(folder +'2002-03.csv')
raw_data_4 = pd.read_csv(folder +'2003-04.csv')
raw_data_5 = pd.read_csv(folder +'2004-05.csv')
raw_data_6 = pd.read_csv(folder +'2005-06.csv')
raw_data_7 = pd.read_csv(folder +'2006-07.csv')
raw_data_8 = pd.read_csv(folder +'2007-08.csv')
raw_data_9 = pd.read_csv(folder +'2008-09.csv')
raw_data_10 = pd.read_csv(folder +'2009-10.csv')
raw_data_11 = pd.read_csv(folder +'2010-11.csv')
raw_data_12 = pd.read_csv(folder +'2011-12.csv')
raw_data_13 = pd.read_csv(folder +'2012-13.csv')
raw_data_14 = pd.read_csv(folder +'2013-14.csv')
raw_data_15 = pd.read_csv(folder +'2014-15.csv')
raw_data_16 = pd.read_csv(folder +'2015-16.csv')
raw_data_17 = pd.read_csv(folder +'2016-17.csv')
raw_data_18 = pd.read_csv(folder +'2017-18.csv')
raw_data_19 = pd.read_csv(folder +'2018-19.csv')
raw_data_20 = pd.read_csv(folder +'2019-20.csv')
raw_data_21 = pd.read_csv(folder +'2020-21.csv')
raw_data_22 = pd.read_csv(folder +'2021-22.csv')

Fetch Gameplay Statistics

In [30]:
#Gets all the statistics related to gameplay
                      
columns_req = ['Date','HomeTeam','AwayTeam','FTHG','FTAG','FTR']

playing_statistics_1 = raw_data_1[columns_req]                      
playing_statistics_2 = raw_data_2[columns_req]
playing_statistics_3 = raw_data_3[columns_req]
playing_statistics_4 = raw_data_4[columns_req]
playing_statistics_5 = raw_data_5[columns_req]
playing_statistics_6 = raw_data_6[columns_req]
playing_statistics_7 = raw_data_7[columns_req]
playing_statistics_8 = raw_data_8[columns_req]
playing_statistics_9 = raw_data_9[columns_req]
playing_statistics_10 = raw_data_10[columns_req]
playing_statistics_11 = raw_data_11[columns_req]   
playing_statistics_12 = raw_data_12[columns_req]
playing_statistics_13 = raw_data_13[columns_req]
playing_statistics_14 = raw_data_14[columns_req]
playing_statistics_15 = raw_data_15[columns_req]
playing_statistics_16 = raw_data_16[columns_req]
playing_statistics_17 = raw_data_17[columns_req]
playing_statistics_18 = raw_data_18[columns_req]
playing_statistics_19 = raw_data_19[columns_req]
playing_statistics_20 = raw_data_20[columns_req]
playing_statistics_21 = raw_data_21[columns_req]
playing_statistics_22 = raw_data_22[columns_req]

Goals Scored and Conceded - End of Matchweek

In [31]:
# Gets the goals scored agg arranged by teams and matchweek
def get_goals_scored(playing_stat):
    # Create a dictionary with team names as keys
    teams = {}
    numeric_cols = playing_stat.select_dtypes(include=['number']).columns
    for i in playing_stat.groupby('HomeTeam')[numeric_cols].mean().T.columns:
        teams[i] = []
    
    # Add goals scored to each team's list
    for i in range(len(playing_stat)):
        HTGS = playing_stat.iloc[i]['FTHG']
        ATGS = playing_stat.iloc[i]['FTAG']
        teams[playing_stat.iloc[i].HomeTeam].append(HTGS)
        teams[playing_stat.iloc[i].AwayTeam].append(ATGS)
    
    # Create a DataFrame with dynamic index based on the maximum matches played
    max_matches = max(len(goals) for goals in teams.values())
    GoalsScored = pd.DataFrame(data=teams, index=[i for i in range(1, max_matches + 1)]).T
    GoalsScored[0] = 0

    # Aggregate goals scored up to that point
    for i in range(2, max_matches + 1):
        GoalsScored[i] = GoalsScored[i] + GoalsScored[i - 1]
    
    return GoalsScored



# Gets the goals conceded agg arranged by teams and matchweek
def get_goals_conceded(playing_stat):
    # Create a dictionary with team names as keys
    teams = {}
    numeric_cols = playing_stat.select_dtypes(include=['number']).columns
    for i in playing_stat.groupby('HomeTeam')[numeric_cols].mean().T.columns:
        teams[i] = []
    
    # Add goals conceded to each team's list
    for i in range(len(playing_stat)):
        ATGC = playing_stat.iloc[i]['FTHG']
        HTGC = playing_stat.iloc[i]['FTAG']
        teams[playing_stat.iloc[i].HomeTeam].append(HTGC)
        teams[playing_stat.iloc[i].AwayTeam].append(ATGC)
    
    # Create a DataFrame with dynamic index based on the maximum matches played
    max_matches = max(len(goals) for goals in teams.values())
    GoalsConceded = pd.DataFrame(data=teams, index=[i for i in range(1, max_matches + 1)]).T
    GoalsConceded[0] = 0

    # Aggregate goals conceded up to that point
    for i in range(2, max_matches + 1):
        GoalsConceded[i] = GoalsConceded[i] + GoalsConceded[i - 1]
    
    return GoalsConceded

def get_gss(playing_stat):
    GC = get_goals_conceded(playing_stat)
    GS = get_goals_scored(playing_stat)

    j = 0
    HTGS = []
    ATGS = []
    HTGC = []
    ATGC = []

    # Use dynamic range based on the size of playing_stat
    for i in range(len(playing_stat)):
        ht = playing_stat.iloc[i].HomeTeam
        at = playing_stat.iloc[i].AwayTeam

        HTGS.append(GS.loc[ht][j])
        ATGS.append(GS.loc[at][j])
        HTGC.append(GC.loc[ht][j])
        ATGC.append(GC.loc[at][j])

        # Increment matchweek after every 10 matches
        if ((i + 1) % 10) == 0:
            j = j + 1

    playing_stat['HTGS'] = HTGS
    playing_stat['ATGS'] = ATGS
    playing_stat['HTGC'] = HTGC
    playing_stat['ATGC'] = ATGC

    return playing_stat


# Apply to each dataset
playing_statistics_1 = get_gss(playing_statistics_1)
playing_statistics_2 = get_gss(playing_statistics_2)
playing_statistics_3 = get_gss(playing_statistics_3)
playing_statistics_4 = get_gss(playing_statistics_4)
playing_statistics_5 = get_gss(playing_statistics_5)
playing_statistics_6 = get_gss(playing_statistics_6)
playing_statistics_7 = get_gss(playing_statistics_7)
playing_statistics_8 = get_gss(playing_statistics_8)
playing_statistics_9 = get_gss(playing_statistics_9)
playing_statistics_10 = get_gss(playing_statistics_10)
playing_statistics_11 = get_gss(playing_statistics_11)
playing_statistics_12 = get_gss(playing_statistics_12)
playing_statistics_13 = get_gss(playing_statistics_13)
playing_statistics_14 = get_gss(playing_statistics_14)
playing_statistics_15 = get_gss(playing_statistics_15)
playing_statistics_16 = get_gss(playing_statistics_16)
playing_statistics_17 = get_gss(playing_statistics_17)
playing_statistics_18 = get_gss(playing_statistics_18)
playing_statistics_19 = get_gss(playing_statistics_19)
playing_statistics_20 = get_gss(playing_statistics_20)
playing_statistics_21 = get_gss(playing_statistics_21)
playing_statistics_22 = get_gss(playing_statistics_22)

C:\Users\97798\AppData\Local\Temp\ipykernel_17264\1764197174.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  playing_stat['HTGS'] = HTGS
C:\Users\97798\AppData\Local\Temp\ipykernel_17264\1764197174.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  playing_stat['ATGS'] = ATGS
C:\Users\97798\AppData\Local\Temp\ipykernel_17264\1764197174.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

Get Respective Points

In [32]:
def get_points(result):
    if result == 'W':
        return 3
    elif result == 'D':
        return 1
    else:
        return 0
    

def get_cuml_points(matchres):
    matchres_points = matchres.applymap(get_points).fillna(0)  # Replace None with 0
    num_weeks = matchres_points.shape[1]
    
    for i in range(2, num_weeks + 1):
        matchres_points[i] = matchres_points[i] + matchres_points[i - 1]
    
    matchres_points.insert(column=0, loc=0, value=[0] * matchres_points.shape[0])
    return matchres_points


def get_matchres(playing_stat):
    # Create a dictionary for match results
    teams = {}
    for team in playing_stat['HomeTeam'].unique():
        teams[team] = []

    for i in range(len(playing_stat)):
        if playing_stat.iloc[i].FTR == 'H':
            teams[playing_stat.iloc[i].HomeTeam].append('W')
            teams[playing_stat.iloc[i].AwayTeam].append('L')
        elif playing_stat.iloc[i].FTR == 'A':
            teams[playing_stat.iloc[i].AwayTeam].append('W')
            teams[playing_stat.iloc[i].HomeTeam].append('L')
        else:
            teams[playing_stat.iloc[i].AwayTeam].append('D')
            teams[playing_stat.iloc[i].HomeTeam].append('D')
    
    # Fill missing matches with None
    max_matches = max(len(results) for results in teams.values())
    for team in teams:
        while len(teams[team]) < max_matches:
            teams[team].append(None)

    return pd.DataFrame(data=teams, index=[i for i in range(1, max_matches + 1)]).T



def get_agg_points(playing_stat):
    matchres = get_matchres(playing_stat)
    cum_pts = get_cuml_points(matchres)
    HTP = []
    ATP = []
    
    for i in range(len(playing_stat)):
        ht = playing_stat.iloc[i].HomeTeam
        at = playing_stat.iloc[i].AwayTeam
        
        # Safely calculate match index
        match_index = min(i // (len(playing_stat) // len(matchres.index)), len(cum_pts.loc[ht]) - 1)
        
        HTP.append(cum_pts.loc[ht].iloc[match_index])
        ATP.append(cum_pts.loc[at].iloc[match_index])
    
    playing_stat['HTP'] = HTP
    playing_stat['ATP'] = ATP
    return playing_stat


# Apply to each dataset
playing_statistics_1 = get_agg_points(playing_statistics_1)
playing_statistics_2 = get_agg_points(playing_statistics_2)
playing_statistics_3 = get_agg_points(playing_statistics_3)
playing_statistics_4 = get_agg_points(playing_statistics_4)
playing_statistics_5 = get_agg_points(playing_statistics_5)
playing_statistics_6 = get_agg_points(playing_statistics_6)
playing_statistics_7 = get_agg_points(playing_statistics_7)
playing_statistics_8 = get_agg_points(playing_statistics_8)
playing_statistics_9 = get_agg_points(playing_statistics_9)
playing_statistics_10 = get_agg_points(playing_statistics_10)
playing_statistics_11 = get_agg_points(playing_statistics_11)
playing_statistics_12 = get_agg_points(playing_statistics_12)
playing_statistics_13 = get_agg_points(playing_statistics_13)
playing_statistics_14 = get_agg_points(playing_statistics_14)
playing_statistics_15 = get_agg_points(playing_statistics_15)
playing_statistics_16 = get_agg_points(playing_statistics_16)
playing_statistics_17 = get_agg_points(playing_statistics_17)
playing_statistics_18 = get_agg_points(playing_statistics_18)
playing_statistics_19 = get_agg_points(playing_statistics_19)
playing_statistics_20 = get_agg_points(playing_statistics_20)
playing_statistics_21 = get_agg_points(playing_statistics_21)
playing_statistics_22 = get_agg_points(playing_statistics_22)

C:\Users\97798\AppData\Local\Temp\ipykernel_17264\340371204.py:11: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  matchres_points = matchres.applymap(get_points).fillna(0)  # Replace None with 0
C:\Users\97798\AppData\Local\Temp\ipykernel_17264\340371204.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  playing_stat['HTP'] = HTP
C:\Users\97798\AppData\Local\Temp\ipykernel_17264\340371204.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  playing_stat['ATP'] = ATP
C:

Get Team Form

In [35]:
def get_form(playing_stat, num):
    form = get_matchres(playing_stat)
    form_final = form.copy()

    # Dynamically calculate the number of matchweeks
    num_weeks = form.shape[1]

    # Create past `num` results for each matchweek
    for i in range(num, num_weeks + 1):
        form_final[i] = ''
        for j in range(num):
            form_final[i] += form[i - j - 1]  # Fetch past results (most recent first)

    return form_final


def add_form(playing_stat, num):
    form = get_form(playing_stat, num)
    num_weeks = form.shape[1]
    num_teams = form.shape[0]

    h = ['M'] * (num * num_teams)
    a = ['M'] * (num * num_teams)

    current_week = num

    for i in range(num * num_teams, len(playing_stat)):
        ht = playing_stat.iloc[i].HomeTeam
        at = playing_stat.iloc[i].AwayTeam

        # Use `.get()` to handle missing keys
        if ht in form.index and at in form.index and current_week < num_weeks:
            past_ht = form.loc[ht][current_week]
            past_at = form.loc[at][current_week]

            h.append(past_ht[num - 1])  # Most recent match result
            a.append(past_at[num - 1])
        else:
            h.append('M')  # Default value
            a.append('M')

        # Increment matchweek index
        if (i + 1) % num_teams == 0:
            current_week += 1

    playing_stat['HM' + str(num)] = h
    playing_stat['AM' + str(num)] = a

    return playing_stat


def add_form_df(playing_statistics):
    for num in range(1, 6):  # Adding form for the last 1 to 5 matches
        playing_statistics = add_form(playing_statistics, num)
    return playing_statistics


# Apply changes dynamically to all datasets
datasets = [playing_statistics_1, playing_statistics_2, playing_statistics_3, playing_statistics_4, 
            playing_statistics_5, playing_statistics_6, playing_statistics_7, playing_statistics_8, 
            playing_statistics_9, playing_statistics_10, playing_statistics_11, playing_statistics_12, 
            playing_statistics_13, playing_statistics_14, playing_statistics_15, playing_statistics_16, 
            playing_statistics_17, playing_statistics_18, playing_statistics_19, playing_statistics_20, 
            playing_statistics_21, playing_statistics_22]

datasets = [add_form_df(dataset) for dataset in datasets]


KeyError: 0